In [1]:
import numpy as np
import pandas as pd
import joblib

from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# 1) Cargar datos
books_df = pd.read_csv("../data/raw/goodreads_data.csv")

# 2) Renombrar columnas
books_df.rename(columns={
    "Book": "book_title",
    "Description": "blurb",
    "Genres": "tags"
}, inplace=True)

# 3) Transformar 'tags' a minúscula y guiones, quedando en un solo string separado por comas
#    Ejemplo: "['Science Fiction', 'Fantasy']" => "science-fiction,fantasy"
books_df["tags"] = books_df["tags"].fillna("[]").apply(
    lambda x: ", ".join(
        tag.strip().lower().replace(" ", "-") for tag in eval(x)
    )
)

# 4) Seleccionar 1000 registros (muestra aleatoria)
books_df = books_df.sample(1000, random_state=42)

# 5) Llenar nulos si quedara alguno y crear columna 'text'
books_df["book_title"] = books_df["book_title"].fillna("")
books_df["blurb"] = books_df["blurb"].fillna("")
books_df["tags"] = books_df["tags"].fillna("")
books_df["text"] = books_df["book_title"] + ". " + books_df["blurb"]

# 6) Convertir la columna 'tags' en listas para MultiLabelBinarizer
#    Ahora 'tags' es un string "science-fiction,fantasy"
#    Lo dividimos por comas (si está vacío, quedará lista vacía)
def parse_comma_tags(s: str):
    return [tag.strip() for tag in s.split(",") if tag.strip()]

books_df["list_tags"] = books_df["tags"].apply(parse_comma_tags)

# 7) Cargar Sentence-BERT y vectorizar
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') 
X = model.encode(books_df['text'].tolist(), show_progress_bar=True)

# 8) Binarizar las etiquetas
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(books_df['list_tags'])

# 9) Separar train y test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 10) Construir el Random Forest en modo multi-output
rf_base = RandomForestClassifier(random_state=42)
multi_rf = MultiOutputClassifier(rf_base)

# 11) Definir la grilla de hiperparámetros
param_grid = {
    'estimator__n_estimators': [5],
    'estimator__max_depth': [5],
    'estimator__min_samples_leaf': [2]
}

# 12) GridSearchCV
grid_search = GridSearchCV(
    multi_rf,
    param_grid=param_grid,
    cv=2, 
    scoring='f1_micro',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor puntuación (CV):", grid_search.best_score_)

best_rf = grid_search.best_estimator_

# 13) Evaluar en test
test_score = best_rf.score(X_test, y_test)
print("Puntuación final en test:", test_score)

# 14) Guardar modelo y binarizador
joblib.dump(best_rf, "../model/book_tagging_rf.joblib")
joblib.dump(mlb, "../model/book_tagging_rf_mlb.joblib")


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
Mejores hiperparámetros: {'estimator__max_depth': 5, 'estimator__min_samples_leaf': 2, 'estimator__n_estimators': 5}
Mejor puntuación (CV): 0.1970418515714289
Puntuación final en test: 0.02


['../model/book_tagging_rf_mlb.joblib']